In [1]:
import os
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
ratios = {
    'ratio1': lambda x: x['boosted_el']/x['boosted_jet'],
    'ratio2': lambda x: (x['boosted_jet']-x['boosted_el'])/x['el_jet'],
    'ratio3': lambda x: x['boosted_jet']-x['boosted_el']
}

In [3]:
wass_distances = pd.read_csv(os.path.join('..', '..', 'data', 'wass_distances.csv'), index_col=0)
for name, func in ratios.items():
    wass_distances[name] = wass_distances.apply(func, axis=1)
wass_distances.head(10)

,name,boosted_el,boosted_jet,el_jet,description,ratio1,ratio2,ratio3
0,reta,0.006093,0.103605,0.098891,fold_0,0.058814,0.986051,0.097512
1,eratio,0.021855,0.570582,0.549355,fold_0,0.038302,0.998858,0.548728
2,f1,0.139836,0.026531,0.113870,fold_0,5.270701,-0.995040,-0.113305
3,f3,0.004670,0.007830,0.012500,fold_0,0.596399,0.252820,0.003160
4,wstot,0.110378,2.799032,2.751361,fold_0,0.039434,0.977209,2.688654
5,weta2,0.000718,0.004058,0.003343,fold_0,0.176922,0.999194,0.003340
6,rhad,0.005016,0.470382,0.473200,fold_0,0.010664,0.983445,0.465366
7,rhad1,0.002919,0.246813,0.249434,fold_0,0.011827,0.977791,0.243894
8,rphi,0.007176,0.109482,0.105540,fold_0,0.065542,0.969359,0.102306
9,reta,0.006228,0.103955,0.098906,fold_1,0.059906,0.988078,0.097727


In [4]:
fold_only = wass_distances[wass_distances['description'].str.startswith('fold')]
statistics = fold_only.groupby('name').agg(['mean', 'std'], numeric_only=True)
statistics.head(10)

/tmp/ipykernel_29329/12137164.py:2: FutureWarning: ['description'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  statistics = fold_only.groupby('name').agg(['mean', 'std'], numeric_only=True)


boosted_el           boosted_jet              el_jet            \
             mean       std        mean       std      mean       std   
name                                                                    
eratio   0.021967  0.000098    0.571166  0.000486  0.549798  0.000324   
f1       0.140010  0.000647    0.026769  0.000622  0.113747  0.000082   
f3       0.004692  0.000025    0.007801  0.000028  0.012493  0.000020   
reta     0.006250  0.000109    0.103909  0.000177  0.098902  0.000050   
rhad     0.005096  0.001533    0.469873  0.001004  0.472929  0.000611   
rhad1    0.003173  0.000554    0.246539  0.000657  0.249258  0.000440   
rphi     0.007205  0.000112    0.109669  0.000185  0.105631  0.000074   
weta2    0.000721  0.000003    0.004060  0.000003  0.003343  0.000001   
wstot    0.113506  0.001882    2.804539  0.003339  2.750798  0.001345   

          ratio1              ratio2              ratio3            
            mean       std      mean       std      mean       std  
name                                                                
eratio  0.038460  0.000154  0.998911  0.000464  0.549199  0.000433  
f1      5.232163  0.096614 -0.995556  0.000726 -0.113242  0.000073  
f3      0.601516  0.004913  0.248832  0.003824  0.003109  0.000048  
reta    0.060147  0.000952  0.987427  0.000872  0.097659  0.000086  
rhad    0.010848  0.003267  0.982759  0.003400  0.464777  0.002063  
rhad1   0.012874  0.002276  0.976356  0.003169  0.243366  0.001178  
rphi    0.065700  0.000945  0.970018  0.000812  0.102464  0.000129  
weta2   0.177657  0.000598  0.998948  0.000273  0.003339  0.000001  
wstot   0.040472  0.000634  0.978274  0.000756  2.691034  0.002142

In [13]:
statistics.sort_values(by=('ratio3', 'mean'), ascending=False)

boosted_el           boosted_jet              el_jet            \
             mean       std        mean       std      mean       std   
name                                                                    
wstot    0.113506  0.001882    2.804539  0.003339  2.750798  0.001345   
eratio   0.021967  0.000098    0.571166  0.000486  0.549798  0.000324   
rhad     0.005096  0.001533    0.469873  0.001004  0.472929  0.000611   
rhad1    0.003173  0.000554    0.246539  0.000657  0.249258  0.000440   
rphi     0.007205  0.000112    0.109669  0.000185  0.105631  0.000074   
reta     0.006250  0.000109    0.103909  0.000177  0.098902  0.000050   
weta2    0.000721  0.000003    0.004060  0.000003  0.003343  0.000001   
f3       0.004692  0.000025    0.007801  0.000028  0.012493  0.000020   
f1       0.140010  0.000647    0.026769  0.000622  0.113747  0.000082   

          ratio1              ratio2              ratio3            
            mean       std      mean       std      mean       std  
name                                                                
wstot   0.040472  0.000634  0.978274  0.000756  2.691034  0.002142  
eratio  0.038460  0.000154  0.998911  0.000464  0.549199  0.000433  
rhad    0.010848  0.003267  0.982759  0.003400  0.464777  0.002063  
rhad1   0.012874  0.002276  0.976356  0.003169  0.243366  0.001178  
rphi    0.065700  0.000945  0.970018  0.000812  0.102464  0.000129  
reta    0.060147  0.000952  0.987427  0.000872  0.097659  0.000086  
weta2   0.177657  0.000598  0.998948  0.000273  0.003339  0.000001  
f3      0.601516  0.004913  0.248832  0.003824  0.003109  0.000048  
f1      5.232163  0.096614 -0.995556  0.000726 -0.113242  0.000073

In [6]:
def get_order(df: pd.DataFrame, sort_col: str) -> pd.Series:
    ascending = sort_col == 'ratio1'
    df = df.sort_values(sort_col, ascending=ascending)
    ordered_ss = df['name'].values
    values = df[sort_col].values
    row = pd.Series(
        [ordered_ss, values],
        index=['order', 'values']
    )
    return row
orders = {
    name: wass_distances.groupby('description').apply(get_order, sort_col=name) # type: ignore
    for name in ratios.keys()
}
orders['ratio2']

,order,values
description,,
complete,"[eratio, weta2, reta, rhad, wstot, rhad1, rphi...","[0.9989697533186417, 0.9989691148718622, 0.987..."
fold_0,"[weta2, eratio, reta, rhad, rhad1, wstot, rphi...","[0.999193529768455, 0.9988577096164344, 0.9860..."
fold_1,"[eratio, weta2, reta, rhad, wstot, rhad1, rphi...","[0.9992039005706516, 0.998955875487918, 0.9880..."
fold_2,"[eratio, weta2, reta, wstot, rhad, rhad1, rphi...","[0.9989988966765141, 0.9986855950147076, 0.987..."
fold_3,"[weta2, eratio, reta, rhad, wstot, rhad1, rphi...","[0.9992413762282488, 0.9981522982926294, 0.987..."
fold_4,"[eratio, weta2, reta, rhad, rhad1, wstot, rphi...","[0.9993444009986554, 0.9986613286773078, 0.988..."


In [7]:
def get_order_differences(df: pd.DataFrame):
    differences = list()
    for left, right in combinations(df.index, 2):
        is_equal = (df.loc[left, 'order'] ==\
            df.loc[right, 'order']).all()   # type: ignore
        if not is_equal:
            differences.append((left, right))
    return differences

order_differences = {
    ratio_name: get_order_differences(ratio_order)
    for ratio_name, ratio_order in orders.items()
}
order_differences

{'ratio1': [],
 'ratio2': [('complete', 'fold_0'),
  ('complete', 'fold_2'),
  ('complete', 'fold_3'),
  ('complete', 'fold_4'),
  ('fold_0', 'fold_1'),
  ('fold_0', 'fold_2'),
  ('fold_0', 'fold_3'),
  ('fold_0', 'fold_4'),
  ('fold_1', 'fold_2'),
  ('fold_1', 'fold_3'),
  ('fold_1', 'fold_4'),
  ('fold_2', 'fold_3'),
  ('fold_2', 'fold_4'),
  ('fold_3', 'fold_4')],
 'ratio3': []}

In [8]:
print(orders['ratio1'].loc['complete', 'order'])
print(orders['ratio1'].loc['fold_3', 'order'])
print(orders['ratio1'].loc['complete', 'order'] == orders['ratio1'].loc['fold_3', 'order'])

['rhad' 'rhad1' 'eratio' 'wstot' 'reta' 'rphi' 'weta2' 'f3' 'f1']
['rhad' 'rhad1' 'eratio' 'wstot' 'reta' 'rphi' 'weta2' 'f3' 'f1']
[ True  True  True  True  True  True  True  True  True]


In [9]:
ratio = 'ratio2'
fold = 'fold_1'
print(orders[ratio].loc['complete', 'order'])
print(orders[ratio].loc[fold, 'order'])
print(orders[ratio].loc['complete', 'order'] == orders[ratio].loc[fold, 'order'])

['eratio' 'weta2' 'reta' 'rhad' 'wstot' 'rhad1' 'rphi' 'f3' 'f1']
['eratio' 'weta2' 'reta' 'rhad' 'wstot' 'rhad1' 'rphi' 'f3' 'f1']
[ True  True  True  True  True  True  True  True  True]


In [10]:
ratio = 'ratio2'
fold = 'fold_2'
print(orders[ratio].loc['complete', 'order'])
print(orders[ratio].loc[fold, 'order'])
print(orders[ratio].loc['complete', 'order'] == orders[ratio].loc[fold, 'order'])

['eratio' 'weta2' 'reta' 'rhad' 'wstot' 'rhad1' 'rphi' 'f3' 'f1']
['eratio' 'weta2' 'reta' 'wstot' 'rhad' 'rhad1' 'rphi' 'f3' 'f1']
[ True  True  True False False  True  True  True  True]


In [11]:
ratio = 'ratio2'
fold = 'fold_3'
print(orders[ratio].loc['complete', 'order'])
print(orders[ratio].loc[fold, 'order'])
print(orders[ratio].loc['complete', 'order'] == orders[ratio].loc[fold, 'order'])

['eratio' 'weta2' 'reta' 'rhad' 'wstot' 'rhad1' 'rphi' 'f3' 'f1']
['weta2' 'eratio' 'reta' 'rhad' 'wstot' 'rhad1' 'rphi' 'f3' 'f1']
[False False  True  True  True  True  True  True  True]
